[Elements of PI](http://elementsofprogramminginterviews.com/solutions/)

In [1]:
from collections import *
from functools import *
from heapq import *
from itertools import *

class BTree:
    def __init__(self, value=None, left=None, right=None, parent=None):
        self.value, self.left, self.right, self.parent = value, left, right, parent

[Longest Subarray of Distinct Entries](https://leetcode.com/articles/longest-substring-without-repeating-characters/): Given a string, find the length of the longest substring without repeating characters.

In [2]:
def longest_subarray_of_distinct_entries(a):
    last_indices = {}
    longest_start, longest_stop = 0, 1
    start = 0
    for stop, e in enumerate(a):
        if e in last_indices:
            if last_indices[e] >= start:
                if stop - start > longest_stop - longest_start:
                    longest_stop, longest_start = stop, start
                start = last_indices[e] + 1
        last_indices[e] = stop
    if len(a) - start > longest_stop - longest_start:
        longest_stop, longest_start = stop, start
    return longest_start, longest_stop

assert (2, 5) == longest_subarray_of_distinct_entries('pwwkew')
assert (3, 9) == longest_subarray_of_distinct_entries('pwwkewabcd')

[Longest Contained Range](https://leetcode.com/problems/longest-consecutive-sequence/description/): Given an array of integers, find the length of the longest consecutive elements sequence.

In [3]:
def longest_contained_range(a):
    q = set(a)
    longest = None
    while q:
        i = q.pop()
        lbound, ubound = i - 1, i + 1
        while lbound in q:
            q.remove(lbound)
            lbound -= 1
        while ubound in q:
            q.remove(ubound)
            ubound += 1
        lbound, ubound = lbound + 1, ubound - 1
        if longest is None or ubound - lbound > longest[1] - longest[0]:
            longest = lbound, ubound
    return longest
            
assert (1, 4) == longest_contained_range([100, 4, 200, 1, 3, 2])

In [4]:
def longest_increasing_subarray(a):
    longest_start, longest_stride = 0, 1  # a tuple of (start and stride), not (start and stop).
    start = 0
    while start < len(a) - longest_stride:
        for start2 in range(start + longest_stride, start, -1):
            if a[start2] <= a[start2 - 1]:
                break
        else:
            start2 = start + longest_stride + 1
            while start2 < len(a) and a[start2] > a[start2 - 1]:
                start2 = start2 + 1
            longest_start, longest_stride = start, start2 - start
        start = start2
    return longest_start, longest_start + longest_stride

assert (3, 7) == longest_increasing_subarray([4, 5, 6, 3, 4, 5, 6, 0, 0, 0, 0, 0, 1, 2, 3])
assert (3, 8) == longest_increasing_subarray([4, 5, 6, 3, 4, 5, 6, 7, 0])
assert (3, 9) == longest_increasing_subarray([4, 5, 6, 3, 4, 5, 6, 7, 8])
assert (4, 9) == longest_increasing_subarray([4, 5, 6, 9, 4, 5, 6, 7, 8])

[Longest Increasing Subsequence](https://leetcode.com/problems/longest-increasing-subsequence/description/): Given an array of integers, find the length of longest increasing subsequence.

In [5]:
def maxima(iterable, key=lambda e: e):
    maxima = []
    for e in iterable:
        if not maxima or key(e) > key(maxima[0]):
            maxima = [e]
        elif key(e) == key(maxima[0]):
            maxima.append(e)
    return maxima

def lis(a):  # also called longest_increasing_subsequence.
    @lru_cache(maxsize=None)
    def lisuff(i):
        cases = (e + (a[i],)
                 for j in range(i)
                 for e in lisuff(j) 
                 if e[-1] <= a[i])
        return maxima(cases, key=len) or [(a[i],)]
    cases = (lisuff(i) for i in range(len(a)))
    return maxima(chain.from_iterable(cases), key=len)

assert [(1, 5, 6), (1, 2, 3)] == lis((7, 8, 1, 5, 6, 2, 3))

[Longest subarray which sums <= K](https://leetcode.com/problems/minimum-size-subarray-sum/description/): Find the longest subarray, which sums <= k.

In [6]:
def longest_subarray_le_k(a, k):
    cumsum = list(accumulate(a))
    cumsum_stop = list(reversed(list(accumulate(reversed(cumsum), min))))
    longest_start, longest_stop = None, None
    start, stop, n = 0, 1, len(cumsum)
    while start < n and stop <= n:
        if start > 0:
            sum_ = cumsum_stop[stop - 1] - cumsum[start - 1]
        else:
            sum_ = cumsum_stop[stop - 1]
        if sum_ <= k:
            if longest_stop is None or stop - start > (longest_stop - longest_start):
                longest_stop, longest_start = stop, start
            stop += 1
        else:
            start += 1
    return longest_start, longest_stop
    
assert (0, 3) == longest_subarray_le_k([10, 20, -30, 40, -10], 10)

In [7]:
def max_rectangle_submatrix(matrix):
    m, n = len(matrix), len(matrix[0])
    HW = namedtuple('hw', ('h', 'w'))
    strides = [[HW(0, 0)] * n for _ in range(m)]

    for r in range(m-1, -1, -1):
        for c in range(n-1, -1, -1):
            if matrix[r][c]:
                strides[r][c] = HW(strides[r + 1][c].h + 1 if r < m - 1 else 1,
                                   strides[r][c + 1].w + 1 if c < n - 1 else 1)

    max_area = 0
    for r in range(m):
        for c in range(n):
            if strides[r][c].w * strides[r][c].h > max_area:
                min_width = strides[r][c].w
                for k in range(strides[r][c].h):
                    min_width = min(min_width, strides[r + k][c].w)
                    max_area = max(max_area, min_width * (k + 1))
    return max_area

matrix = [  # 6 x 5
    [1, 0, 1, 1, 1],
    [1, 1, 1, 1, 1],
    [1, 1, 1, 1, 0],
    [0, 1, 1, 1, 1],
    [1, 1, 1, 1, 1],
    [1, 1, 1, 0, 1]
]
assert 12 == max_rectangle_submatrix(matrix)

In [8]:
def largest_k_from_max_heap(max_heap, k):
    largest = [(-max_heap[0], 0)]  # the largest a[0] and the index (0).
    for _ in range(k):
        i = largest[0][1]
        yield -heappop(largest)[0]
        for k in (2 * i + j for j in range(1, 3)):
            if k < len(max_heap):
                heappush(largest, (-max_heap[k], k))

max_heap = [
          561, 
       314,   401, 
    28, 156, 359, 271,
  11, 3]
assert [561, 401, 359, 314] == list(largest_k_from_max_heap(max_heap, 4))

In [9]:
def islice(tree, start=0, stop=None, step=None):
    e, i, stack = tree, 0, []
    while (e or stack) and (stop is None or i < stop):
        if e:
            stack.append(e)
            e = e.left
        else:
            e = stack.pop()
            if start <= i and (i - start) % step == 0:
                yield e
            e, i = e.right, i + 1

def irslice(tree, start, stop=None, step=None):  # step not supported yet.
    e, i, stack = tree, 0, []
    while (e or stack) and (stop is None or i < stop):
        if e:
            stack.append(e)
            e = e.right
        else:
            e = stack.pop()
            if start <= i and (step is None or (i - start) % step == 0):
                yield e
            e, i = e.left, i + 1

# tree:   4
#       /  \
#     3     8
#    /     /  \
#   2     5    9
#  1       7
# 0       6
tree = BTree(4, 
             BTree(3, BTree(2, BTree(1, BTree(0)))),
             BTree(8, BTree(5, None, BTree(7, BTree(6))), BTree(9)))

assert [0, 3, 6, 9] == [e.value for e in islice(tree, 0, stop=10, step=3)]
assert [0, 3, 6] == [e.value for e in islice(tree, 0, stop=9, step=3)]
assert [0, 3, 6] == [e.value for e in islice(tree, 0, stop=7, step=3)]
assert [0, 3] == [e.value for e in islice(tree, 0, stop=6, step=3)]
assert [9, 8] == [e.value for e in irslice(tree, 0, stop=2)]
assert [8, 7] == [e.value for e in irslice(tree, 1, stop=3)]

In [10]:
def islice(tree, start=0, stop=None, step=None):
    prev = next = None
    parents = {}
    while tree:
        if prev is parents.get(tree):
            for e in filter(None, (tree.left, tree.right)):
                parents[e] = tree                
            if tree.left:
                next_ = tree.left
            else:
                yield tree.value
                next_ = tree.right or parents.get(tree)
        elif prev is tree.left:
            yield tree.value
            next_ = tree.right or parents.get(tree)
        else:
            assert prev is tree.right
            next_ = parents.get(tree)
        prev, tree = tree, next_
    
assert [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] == list(islice(tree))

[Largest Rectangle in Histogram](https://leetcode.com/problems/largest-rectangle-in-histogram/description/): Given n non-negative integers representing the histogram's bar height where the width of each bar is 1, find the area of largest rectangle in the histogram.

| Histogram | Area |
|:---------:|:----:|
| ![histogram](https://leetcode.com/static/images/problemset/histogram.png) | ![histogram-area](https://leetcode.com/static/images/problemset/histogram_area.png) |

In [11]:
def largest_rect_area(heights):
    in_process = []
    largest_area = 0
    for stop, h in enumerate(heights + [0]):
        while in_process and h < in_process[-1][1]:
            start, height = in_process.pop()
            width = stop - start
            largest_area = max(largest_area, height * width)
        in_process.append((stop, h))
    return largest_area

assert 10 == largest_rect_area([2, 1, 5, 6, 2, 3])
assert 21 == largest_rect_area([3, 4, 5, 5, 5, 4, 3, 2, 1])

[Compute Skyline](https://leetcode.com/problems/the-skyline-problem/description/): the buildings in Figure A are recorded as: [ [2 9 10], [3 7 15], [5 12 12], [15 20 10], [19 24 8] ]. The skyline in Figure B should be represented as:[ [2 10], [3 15], [7 12], [12 0], [15 10], [20 8], [24, 0] ].


| Building | Skyline |
|:--------:|:-------:|
| ![x](http://leetcode.com/static/images/problemset/skyline1.jpg "title1")>        | ![y](http://leetcode.com/static/images/problemset/skyline2.jpg "titlt2") |

In [12]:
from collections import defaultdict, namedtuple
from heapq import heappop, heappush

class Buliding:
    def __init__(self, height):
        self.height = height
        self.processed = False
        
    def __lt__(self, other):
        return other.height < self.height

Event = namedtuple('Event', 'beginning, ending')

def skyline(buildings):
    events = defaultdict(lambda: Event(beginning=[], ending=[]))
    for left, height, right in buildings:
        b = Buliding(height)
        events[left].beginning.append(b)
        events[right].ending.append(b)

    in_process = []
    last_h = 0
    for i, event in sorted(events.items()):
        for b in event.beginning:
            heappush(in_process, b)
        for b in event.ending:
            b.processed = True
        while in_process and in_process[0].processed:
            heappop(in_process)
        
        h = in_process[0].height if in_process else 0
        if h != last_h:
            yield i, h
            last_h = h

buildings = [[2, 9, 10], [3, 7, 15], [5, 12, 12], [15, 20, 10], [19, 24, 8]]
assert [(2, 9), (5, 12), (12, 7), (15, 0)] == list(skyline(buildings))

In [13]:
def skyline(buildings):
    start_i = min(b[0] for b in buildings)
    stop_i = 2 + max(b[2] for b in buildings)
    heights = [0] * (stop_i - start_i)
    for b in buildings:
        for j in range(b[0] - start_i, b[2] - start_i + 1):
            heights[j] = max(heights[j], b[1])
    skyline = [[start_i, heights[0]]]
    for j in range(1, len(heights)):
        if heights[j] > heights[j - 1]:
            skyline.append([start_i + j, heights[j]])
        elif heights[j] < heights[j - 1]:
            skyline.append([start_i + j - 1, heights[j]])
    return skyline

buildings = [[2, 9, 10], [3, 7, 15], [5, 12, 12], [15, 20, 10], [19, 24, 8]]
assert [[2, 9], [5, 12], [12, 7], [15, 0]] == skyline(buildings)

[Rain water](https://leetcode.com/problems/trapping-rain-water/description/): Given n non-negative integers representing an elevation map where the width of each bar is 1, compute how much water it is able to trap after raining. For example, given [0,1,0,2,1,0,1,3,2,1,2,1], return 6.

![](http://www.leetcode.com/static/images/problemset/rainwatertrap.png)

In [14]:
def rain_water(elevations):
    def rain_water_closed_end(elevations):
        max_elevation = 0
        volume = 0
        for e in elevations:
            if max_elevation < e:
                max_elevation = e
            else:
                volume += max_elevation - e
        return volume

    max_i, _ = max(enumerate(elevations), key=lambda q: q[1])
    return (rain_water_closed_end(elevations[:max_i]) +
        rain_water_closed_end(reversed(elevations[max_i+1:])))

assert 6 == rain_water([0, 1, 0, 2, 1, 0, 1, 3, 2, 1, 2, 1])

In [15]:
def justify(words, max_line_length, fillchar='_'):
    result, start, word_length_sum = [], 0, 0
    for stop, word in enumerate(words):
        if word_length_sum + len(word) + stop - start > max_line_length:
            for i in range(max_line_length - word_length_sum):
                offset = i % (stop - start - 1 or 1)
                words[start + offset] += fillchar
            result.append(''.join(words[start:stop]))
            start, word_length_sum = stop, 0
        word_length_sum += len(word)
    return result + [fillchar.join(words[start:]).ljust(max_line_length, fillchar)]

assert (['This____is____an', 'example__of_text', 'justification.__']
  == justify(['This', 'is', 'an', 'example', 'of', 'text', 'justification.'], 16))

In [16]:
class Trie:
    def __init__(self, value=None):
        self.children, self.value = {}, value

    def __setitem__(self, k, v):
        if len(k) > 1:
            if k[0] not in self.children:
                self.children[k[0]] = Trie()
            self.children[k[0]][k[1:]] = v
        else:
            self.children[k[0]] = v

    def __getitem__(self, k):
        if len(k) > 0:
            if k[0] in self.children:
                return self.children[k[0]][k[1:]]
        else:
            return self

    def values(self):
        if self.value is not None:
            yield self.value
        for child in self.children.values():
            yield from child.values()

s, tree = "bananas", Trie()
for i, _ in enumerate(s):
    tree[s[i:]] = Trie(i)

# all indices of query strings
def indices_of(q):
    trie = tree[q]
    return list(trie.values()) if trie else []

q = "ba na nas s bas".split(" ")
assert [[0], [2, 4], [4], [6], []] == [sorted(indices_of(e)) for e in q]

18.5 Given a large text file containing words, find the shortest distance between two or k words in terms of the number of words. What is the time and space complexity?

In [17]:
from collections import deque
import heapq

def min_word_window(positions): # e.g., [[0, 89, 130], [95, 123, 177, 199], [70, 105, 117]], O(L*logK)
    positions = [deque(e) for e in positions]
    min_window = window = [e.popleft() for e in positions] # [0, 95, 70]
    heap = []
    for i, e in enumerate(window):
        heapq.heappush(heap, (e, i)) # (position e, i).
    while positions[heap[0][1]]: # positions[i] is non-empty.
        i = heapq.heappop(heap)[1]
        window[i] = positions[i].popleft()
        min_window = min(min_window, window, key=lambda w: max(w) - min(w))
        heapq.heappush(heap, (window[i], i))
    return (min(min_window), max(min_window))

assert (117, 130) == min_word_window([[0, 89, 130], [95, 123, 177, 199], [70, 105, 117]])

12.7. Find the smallest subarray covering all values.

In [18]:
def pangram(text='aabbaacxxx', alphas='abc'):
    shortest = None
    counter = Counter(set(alphas))
    coverage = 0
    start = 0
    for end, c in enumerate(text):
        if c in alphas:
            counter[c] -= 1
            if counter[c] == 0:
                coverage += 1
        while coverage == len(alphas):
            if shortest is None or end - start < shortest[1] - shortest[0]:
                shortest = (start, end)
            c = text[start]
            if c in alphas:
                counter[c] += 1
                if counter[c] == 1:
                    coverage -= 1
            start += 1
    return shortest

assert (0, 2) == pangram('abcybbcabc', 'abc')

5.11 Compute the next permutation.

6.13 Find the first occurence of a substring.

9.2 Test if a binary tree is symmetric.

In [19]:
def is_symmetric(l, r):
    if not any((l, r)):
        return True
    if not all((l, r)):
        return False
    return all((l.value == r.value,
                is_symmetric(l.left, r.right),
                is_symmetric(l.right, r.left)))

# tree:   1
#       2   2
#     3       3
tree = BTree(1, BTree(2, BTree(3)), BTree(2, None, BTree(3)))
assert is_symmetric(tree.left, tree.right)

9.5 Sum the root-to-leaf paths in a binary tree.

In [20]:
def sum_to_leaf(tree, cumsum=0):  # cumulative sum.
    if tree:
        cumsum = 2 * cumsum + tree.value
        if not any((tree.left, tree.right)):
            yield cumsum
        else:
            yield from sum_to_leaf(tree.left, cumsum)
            yield from sum_to_leaf(tree.right, cumsum)

# tree:   1
#       0   1
#          1
tree = BTree(1, BTree(0), BTree(1, BTree(1)))
assert 9 == sum(sum_to_leaf(tree))  # B10(2) + B111(7)

9.6 Find a root to leaf path with specified sum.

In [21]:
# tree: -1
#          3
#        -1
#       2  3
tree = BTree(-1, None, BTree(3, BTree(-1, BTree(2), BTree(3))))

def paths_to_leaves_of_sum(tree, sum_, breadcrumbs=None):
    if tree:
        sum_ -= tree.value
        breadcrumbs = breadcrumbs or []
        breadcrumbs.append(tree.value)
        if not any((tree.left, tree.right)) and sum_ == 0:
            yield breadcrumbs[:]
        yield from paths_to_leaves_of_sum(tree.left, sum_, breadcrumbs)
        yield from paths_to_leaves_of_sum(tree.right, sum_, breadcrumbs)
        breadcrumbs.pop()

assert ([-1, 3, -1, 2],) == tuple(paths_to_leaves_of_sum(tree, 3))

4.9 Given a binary tree in which each node contains a value. Design an algorithm to print all paths, which sum to a given value. Note that a path can start or end anywhere in the tree.

In [22]:
# tree: -1
#          3
#        -1
#       2  3
tree = BTree(-1, None, BTree(3, BTree(-1, BTree(2), BTree(3))))

breadcrumbs = []
def paths_of_sum(tree, sum_, breadcrumbs=None, sums=None, indices=None):
    if tree:
        breadcrumbs = breadcrumbs or []
        sums = sums or []
        indices = indices or defaultdict(list, ((sum_, [0]),))
        breadcrumbs.append(tree.value)
        sums.append(tree.value + (sums[-1] if sums else 0))
        for start in indices[sums[-1]]:
            yield breadcrumbs[start:]
        indices[sums[-1] + sum_].append(len(breadcrumbs))
        yield from paths_of_sum(tree.left, sum_, breadcrumbs, sums, indices)
        yield from paths_of_sum(tree.right, sum_, breadcrumbs, sums, indices)
        indices[sums[-1] + sum_].pop()
        sums.pop()
        breadcrumbs.pop()

assert ([-1, 3], [3, -1], [2], [-1, 3]) == tuple(paths_of_sum(tree, 2))
assert ([-1], [-1]) == tuple(paths_of_sum(tree, -1))
assert ([-1, 3, -1], [-1, 2]) == tuple(paths_of_sum(tree, 1))

9.15 Compute the exterior of a binary tree.

In [23]:
def is_leaf(tree):
    return not any((tree.left, tree.right))

def left_boundary_leaves(tree, exterior):
    if tree:
        if exterior or is_leaf(tree):
            yield tree.value
        yield from left_boundary_leaves(tree.left, exterior)
        yield from right_boundary_leaves(tree.right, False)

def right_boundary_leaves(tree, exterior):
    if tree:
        if exterior or is_leaf(tree):
            yield tree.value
        yield from right_boundary_leaves(tree.right, exterior)
        yield from left_boundary_leaves(tree.left, False)

def exterior(tree):
    if tree:
        yield tree.value
        yield from left_boundary_leaves(tree.left, True)
        yield from reversed(list(right_boundary_leaves(tree.right, True)))

# tree:     a
#         /   \
#       b      f
#     c  e   g  h
#   d       i   

d = BTree('d')
c = BTree('c', d)
e = BTree('e')
b = BTree('b', c, e)
a = BTree('a', b, BTree('f', BTree('g', BTree('i')), BTree('h')))

assert ['a', 'b', 'c', 'd', 'e', 'i', 'h', 'f'] == list(exterior(a))

10. 4 Compute the k closest starts.

In [24]:
def power_set(a, i=0):
    if i == len(a):
        yield tuple()
    else:
        for e in power_set(a, i+1):
            yield from ((a[i],) + e, e)

assert [(1, 2, 3), (2, 3), (1, 3), (3,), (1, 2), (2,), (1,), ()] == list(power_set([1, 2, 3]))

15.1 Write a program that solves Tower of Hanoi puzzle of N disks. http://wikipedia.org/wiki/Tower_of_Hanoi

In [25]:
def move_tower(source, target, spare, n):
    if n > 0:
        yield from move_tower(source, spare, target, n-1)
        yield (source, target)
        yield from move_tower(spare, target, source, n-1)

assert [
    ('A', 'C'),
    ('A', 'B'),
    ('C', 'B'),
    ('A', 'C'),
    ('B', 'A'),
    ('B', 'C'),
    ('A', 'C')
] == list(move_tower('A', 'C', 'B', 3))

15.2 Generate non-attacking placements of queens. 

In [26]:
def peaceful_at(queens, r, c): # Peaceful at row and column (r, c)?
    return all([c0 != c and r - r0 != abs(c - c0)
                for r0, c0 in enumerate(queens)])

def queens_in_peace(n, queens):
    r = len(queens)  # we're at row r.
    if r == n:
        yield queens[:]
    else:
        for c in range(n):
            if peaceful_at(queens, r, c):
                queens.append(c)
                yield from queens_in_peace(n, queens)
                queens.pop()

assert [1, 3, 0, 2] == next(queens_in_peace(4, []))
assert [[0], [], []] == list(next(queens_in_peace(n, []), [])
                             for n in range(1, 4))

15.3 Generate permutations.

In [27]:
def permutation(a, i=0): # places uniq values at index i.
    if i == len(a):
        yield a
    else:
        uniq_value_indices = {a[j]: j for j in range(i, len(a))}.values()
        for j in uniq_value_indices:
            a[i], a[j] = a[j], a[i]
            yield from permutation(a, i+1)
            a[i], a[j] = a[j], a[i]

assert {'aab', 'aba', 'baa'} == set("".join(e) for e in permutation(list("aab")))
assert {(1, 1, 2), (1, 2, 1), (2, 1, 1)} == set(tuple(e) for e in permutation([1, 1, 2]))

15.4 Generate the power set.

In [28]:
def power_set(a, start=0):
    if start == len(a):
        yield ()
    else:
        for e in power_set(a, start+1):
            yield from (e, (a[start],) + e)

assert ({(), ('a',), ('b',), ('c',), ('a', 'b'), ('a', 'c'), ('b', 'c'), ('a', 'b', 'c')}
    == set(power_set(list("abc"))))

15.5 Generate all subsets of size k.

In [29]:
def combination(a, k, i=0):
    if k == 0:
        yield set()
    elif i < len(a):
        for e in combination(a, k-1, i+1):
            yield {a[i]} | e
        yield from combination(a, k, i+1)

assert 10 == len(list(combination(range(5), 2)))
assert 10 == len(list(combination(range(5), 3)))

15.6 Generate strings of matched parens.

In [30]:
def combine_parens(o, c):  # open and close counts.
    if c == 0:
        yield ""
    elif o == 0:
        yield ")" * c
    elif c == o:
        yield from ("(" + e for e in combine_parens(o-1, c))
    else:
        yield from ("(" + e for e in combine_parens(o-1, c))
        yield from (")" + e for e in combine_parens(o, c-1))

assert {'((()))', '(()())', '(())()', '()(())', '()()()'} == set(combine_parens(3, 3))

15.7 Generate palindromic decompositions.

In [31]:
def palindromic_partitions(s):
    if len(s) < 2:
        yield (s,)
    else:
        for k in range(1, len(s)):
            if s[:k] == s[k-1::-1]:
                for e in palindromic_partitions(s[k:]):
                    yield (s[:k],) + e

assert {('a', 'a', 'b'), ('aa', 'b')} == set(palindromic_partitions('aab'))

15.8 Generate binary trees.

In [32]:
def binary_trees(n):
    if n == 0:
        yield None
    for l in range(n):
        r = n - 1 - l
        l_trees = binary_trees(l)
        r_trees = binary_trees(r)
        for e_l, e_r in product(l_trees, r_trees):
            yield BTree(None, e_l, e_r)

ilen = lambda iterable: sum(1 for _ in iterable)
assert [1, 1, 2, 5, 14] == [ilen(binary_trees(e)) for e in range(5)]

15.9 Write a program that solves a sudoku puzzle.

15.10 Write a program that takes n as input and returns an n-bit Grey code.

In [33]:
def acceptable(candidate, j):
    i = candidate[-1]
    k = i ^ j
    return k and (k & k - 1) == 0

def grey_code(n, candidate=None, accepted=None):
    candidate = candidate or []
    accepted = accepted or set()
    if len(candidate) == 2**n:
        if acceptable(candidate, candidate[0]):
            yield candidate[:]
    else:
        for e in range(2**n):
            if not accepted or e not in accepted and acceptable(candidate, e):
                candidate.append(e)
                accepted.add(e)
                yield from grey_code(n, candidate, accepted)
                accepted.remove(e)
                candidate.pop()

gc = grey_code(2)
assert [0, 1, 3, 2] == next(gc, None)
assert 7 == len(tuple(gc))

In [34]:
def grey_code(n):
    if n == 1:
        yield from ((0, 1), (1, 0))
    for e in grey_code(n-1):
        yield e
    for e in reversed(grey_code(n-1)):
        yield ee

15.11 Compute the diameter of a tree.

In [35]:
def parse(preorder, inorder):
    if preorder:
        v = preorder[0]
        k = inorder.index(v)
        return BTree(v,
            parse(preorder[1:k+1], inorder[:k]),
            parse(preorder[k+1:], inorder[k+1:]))

DH = namedtuple('DiameterAndHeight', ('diameter', 'height'))

def diameter(tree):  # returns DH(diameter, height)
    if tree is None:
        return DH(0, -1)  # leaf has (0, 0)
    l = diameter(tree.left)
    r = diameter(tree.right)
    h = 1 + max(l.height, r.height)
    d = max(l.diameter, r.diameter, l.height + r.height + 2)
    return DH(d, h)

# tree input:   a
#             b
#          c    f (1, 1)
#           d     g (0, 0)
#            e
tree = parse('abcdefg', 'cdebfga')
assert (5, 4) == diameter(tree)

Interval Covering: Given a set of closed intervals, design an efficient algorithm that finds a minimum size set of numbers that covers all the intervals.

In [36]:
def find_minimum_point_set(intervals):
    intervals.sort(key=lambda interval: interval[1])
    points = []
    for e in intervals:
        if not points or points[-1] < e[0]:
            points.append(e[1])
    return points

assert [3, 9] == find_minimum_point_set([[0, 3], [2, 6], [3, 4], [6, 9]])

[Insert Interval](https://leetcode.com/problems/insert-interval/description/): Given a set of non-overlapping intervals, insert a new interval into the intervals (merge if necessary). You may assume that the intervals were initially sorted according to their start times.

In [37]:
Interval = namedtuple('Interval', 'left, right')

def insert_interval(intervals, new_interval):
    result = []
    i = 0
    while (i < len(intervals) and
        new_interval.left > intervals[i].right):
        result.append(intervals[i])
        i += 1
    while (i < len(intervals) and
           new_interval.right >= intervals[i].left):
        new_interval = Interval(
            min(new_interval.left, intervals[i].left),
            max(new_interval.right, intervals[i].right))
        i += 1
    return result + [new_interval] + intervals[i:]

intervals = [Interval(*e) for e in [[1,2],[3,5],[6,7],[8,10],[12,16]]]
assert [(1,2),(3,10),(12,16)] == insert_interval(intervals, Interval(4,9))

[Merge Interval](https://leetcode.com/problems/merge-intervals/description/): Given a collection of intervals, merge all overlapping intervals. Intervals may be open or closed at either end.

In [38]:
Endpoint = namedtuple('Endpoint', 'x, is_closed')

class Interval:
    def __init__(self, left, right):
        self.left, self.right = left, right

    def __lt__(self, other):
        return (self.left.x < other.left.x
                or self.left.x == other.left.x
                   and self.left.is_closed and not other.left.is_closed)

def merge_intervals(intervals):
    intervals.sort()
    result = [intervals[0]]
    for e in intervals:
        if intervals and (e.left.x < result[-1].right.x
                          or e.left.x == result[-1].right.x
                             and e.left.is_closed or e.right.is_closed):
            if (e.right.x > result[-1].right.x
                or e.right.x == results[-1].right.x
                   and e.right.is_closed):
                result[-1].right = e.right
        else:
            result.append(i)
    return result